In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from nltk.tokenize import word_tokenize
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.layers import Layer
import gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
#Preprocessing function
def preprocessing(data_frame):
    ## Preprocessing
    # Removing URLs whithin the tweets
    data_frame["Text"] = data_frame["Text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    # Removing emails, hashtags and punctuations
    data_frame['Text'] = data_frame["Text"].str.replace(r'\S*@\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'#\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'[^\w\s]+', ' ').str.strip()

    # Lowercase Text
    data_frame['Text'] = data_frame['Text'].str.lower()

    # # Removing stopwords
    stop = stopwords.words('english')
    data_frame['Text'].apply(lambda x: [item for item in str(x) if item not in stop])

    # Removing newline characters
    data_frame['Text'] = data_frame['Text'].str.rstrip()

    # Tokenizing Posts and counting the length of each post
    data_frame['Tokens'] = data_frame.apply(lambda row: word_tokenize(str(row['Text'])), axis=1)
    data_frame['Length'] = data_frame.apply(lambda row: len(row['Tokens']), axis=1)

    return data_frame

In [ ]:
# a custome Attention layer
class Attention(Layer):
  def __init__ (self, return_sequences=True):
    self.return_sequences = return_sequences
    super(Attention, self).__init__()

  def build (self, input_shape):
    self.W = self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal")
    self.b = self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
    super(Attention, self).build(input_shape)

  def call(self, x):
    e = K.tanh(K.dot(x, self.W)+self.b)
    a = K.softmax(e, axis=1)
    output = x*a
    if self.return_sequences:
      return output
    return K.sum(output, axis = 1)


# Data

In [ ]:
# ## Preparing the data
# twitter_path = "/content/drive/MyDrive/MH ML project/Datasets/suicidal-tendency-tweets.csv"

# ## reading the abovementioned datasets
# df = pd.read_csv(twitter_path, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
# df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})

# df = preprocessing(df)
# df = df.drop('Text', axis=1)
# df

In [ ]:
twitter_path = "/content/drive/MyDrive/MH ML project/Datasets/suicidal-tendency-tweets.csv"
twitter_only_ones = "/content/drive/MyDrive/MH ML project/Datasets/Twitter/suicideTweetData_56000.csv"

## reading the abovementioned datasets
df = pd.read_csv(twitter_path, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})
ones =  list(df['Label']).count(1)
zeroes = list(df['Label']).count(0)
total = len(df['Label'])
required_ones = np.absolute(zeroes-ones)
print(total, ones, zeroes, required_ones)

more_ones = pd.read_csv(twitter_only_ones, encoding='latin-1',  nrows = required_ones)
more_ones = more_ones.rename(columns={'Tweet': 'Text', 'label': 'Label'})
more_ones['Label'] = more_ones['Label'].apply(lambda x: 1)
more_ones

In [ ]:
df = df.append(more_ones, ignore_index=True)

In [ ]:
df

In [ ]:
ones =  list(df['Label']).count(1)
zeroes = list(df['Label']).count(0)
total = len(df['Label'])
required_ones = np.absolute(zeroes-ones)
print(total, ones, zeroes, required_ones)

In [ ]:
df = preprocessing(df)

# Word2Vec embedding

In [ ]:
# word2vec embedding
wd2vc_model = gensim.models.Word2Vec(df['Tokens'], min_count = 1, size = 300, window = 5)
wd2vc_model.train(df['Tokens'],epochs=10,total_examples=len(df['Tokens']))

(5837975, 7617330)

In [ ]:
# words most similar to a given word.
similar_words = wd2vc_model.wv.most_similar('kill')
print(similar_words)

In [ ]:
## Creating the embedding matrix
vocab = wd2vc_model.wv.vocab
print("The total number of words are : ", len(vocab))
vocab = list(vocab.keys())

word_vec_dict = {}
for word in vocab:
    word_vec_dict[word] = wd2vc_model.wv.get_vector(word)
print("The no of key-value pairs : ", len(word_vec_dict))

max_length = max(df['Length'])
print("maximum length = ", max_length)

tok = Tokenizer()
tok.fit_on_texts(df['Tokens'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['Tokens'])

embed_dim = 300
pad_rev = pad_sequences(encd_rev, maxlen=max_length, padding='post')
print("pad_shape = ", pad_rev.shape)

# now creating the embedding matrix
embed_matrix = np.zeros(shape=(vocab_size, embed_dim))
print(embed_matrix.shape)
for word, i in tok.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

# Data Preparation

In [ ]:
## Spliting the data
X, X_test, y, y_test = train_test_split(pad_rev, df['Label'], shuffle=True, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=33)

print("Train = ", X_train.shape, y_train.shape)
print("Test = ", X_test.shape, y_test.shape)
print("Validation = ",X_val.shape, y_val.shape)

# Model

In [ ]:
# ensemble learning model proposed in the paper
## LSTM and CNN recurrent learning module: Building the model

from keras.initializers import Constant

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_length,
                                    embeddings_initializer=Constant(embed_matrix)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.LSTM(units=100, return_sequences=True))
model.add(Attention(return_sequences=True))
model.add(tf.keras.layers.Conv1D(3, (8,), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4, activation='softmax'))
model.summary()


# Compiling and Executing the model

In [ ]:
## Compiling and fitting
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
#Prediction on the test data
pred = model.predict(X_test)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
  y_pred[item] = list(pred[item]).index(max(pred[item]))

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)